In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = (SparkSession.builder.master("local").appName("stock-price-streaming") \
        .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.11:2.4.5,org.mongodb.spark:mongo-spark-connector_2.11:2.4.0") \
        .getOrCreate())


In [15]:
# Amazon consumer:
amazon_df = spark.readStream.format("kafka")\
            .option("kafka.bootstrap.servers", "kafka:9092") \
            .option("subscribe", "amazon-topic")\
            .load()

# Convert the binaries into string:

amazon_df_string = amazon_df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")


# Create a temporary view for running sql queries to print data into the console:
amazon_df_string.createOrReplaceTempView("amz_message")



# Print the output in the console for testing purpose:

amz_res = spark.sql("select * from amz_message")
amz_res.writeStream.format("console").outputMode("append").start()

In [23]:
# Apple consumer:

apple_df = spark.readStream.format("kafka").option("kafka.bootstrap.servers", "kafka:9092")\
           .option("subscribe", "apple-topic").load()

# Convert the data to string:
apple_df_string = apple_df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")

# Create a temporary view:
apple_df_string.createOrReplaceTempView("apple_message")

# Print the data in the console:
apple_res = spark.sql("Select * from apple_message")
apple_res.writeStream.format("console").outputMode("append").start()

In [17]:
# Coke consumer:

coke_df = spark.readStream.format("kafka").option("kafka.bootstrap.servers", "kafka:9092") \
          .option("subscribe", "coke-topic").load()


# Convert the data back to string:
coke_df_string = coke_df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")

# Create a temporary view:
coke_df_string.createOrReplaceTempView("coke_message")


# Print the output in the console:
coke_res = spark.sql("Select * from coke_message")
coke_res.writeStream.format("console").outputMode("append").start()

In [ ]:
# Consumer for Google:

google_df = spark.readStream.format("kafka").option("kafka.bootstrap.servers", "kafka:9092") \
            .option("subscribe", "google-topic").load()

# Convert the data back to string:
google_df_string = google_df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")

# Create a temporary view:
google_df_string.createOrReplaceTempView("google_message")

# Print the output in the console:
google_res = spark.sql("Select * from google_message")
google_res.writeStream.format("console").outputMode("append").start()


In [ ]:
def consumer_func(topic, )